Grading for pre-lecture questions is all or nothing. Partial credit is available for in-class assignments and checkpoints. Partial credit will primarily be determined by the quality of the comments you make in your code. Tell me what you intend the code to do. If there are no comments, there is no partial credit!

In [ ]:
NAME = ""
COLLABORATORS = ""

---

# Learning Objectives

This lecture will show you how to:
1. Evaluate numerical integrals with the trapezoidal rule and Simpson's rule
2. Estimate the error of your result
3. Understand the method behind Gaussian quadrature
4. Integrate over infinite ranges
5. Evaluate multiple integrals

In [ ]:
# imports
import numpy as np
import matplotlib.pyplot as plt

from scipy import integrate # integration routines

import grading_helper as _test

# Trapezoidal Rule and Simpson's Rule

Update: As of scipy 1.6.0, `trapz` has been renamed `trapezoid` and `simps` has been renamed `simpson`. Apparently "simps" makes you young people giggle. The old names still work, and you'll se them older code.

In [ ]:
%video qv6SwkYKF3w

Summary:

- The Trapezoidal rule approximates an integrand as a series of line segments.
- Simpson's rule is similar, but uses quadratic curves.
- Both the Trapezoidal rule and Simpson's rule are the first few terms in the **Taylor expansion** of the integrand.
- `scipy.integrate.trapezoid(array)` and `scipy.integrate.simpson(array)` apply this technique.

## Your Turn

Write a function named `trapezoidal(f, a, b, N)` that takes a function of one variable `f`, integration limits `a` and `b`, and number of steps `N`, and returns the approximate integral using the trapezoidal rule.

You are not permitted to use `scipy`. You need to code the algorithm yourself (it doesn't require that much code).

In [ ]:
%%graded # 2 points

# YOUR CODE HERE

In [ ]:
%%tests

def _f1(x): return x # should be exact for trapezoial rule
def _f2(x): return x**2
def _f3(x): return x**3

_test.code_contains("def trapezoidal")
assert trapezoidal is not integrate.trapezoid
_test.similar(trapezoidal(_f1, 0, 1, 2), 1/2) #
_test.similar(trapezoidal(_f2, 0, 1, 2), 1/2) # with only 2 points these all look the same to trapezoidal rule
_test.similar(trapezoidal(_f3, 0, 1, 2), 1/2) #
_test.similar(trapezoidal(_f2, 0, 1, 1000), 1/3)  #
_test.similar(trapezoidal(_f3, 0, 1, 1000), 1/4)  # more accurate with more points
_test.similar(trapezoidal(_f2, 2, 4, 1000), 56/3) #

# Function Decorators

In [ ]:
%video 8iS7DQwkfwM

Summary:

- Python has a special type of function called a **decorator**, which is a function that modifies another function.
- Use by adding `@decorator_name` to the line before the function definition.
- We can use the `@np.vectorize` decorator to quickly write a function that works seamlessly with numpy arrays.

# Estimating Error

In [ ]:
%video RoNIpeYH6mA

Summary:

- The Trapezoidal rule has errors that scale like $\epsilon\propto N^{-2}\,,$
while Simpons's rule has errors that scale like $\epsilon\propto N^{-4}\,.$
- Simposn's rule is *usually* more accurate, with the exception being functions that vary rapidly between points.
- Since floating point numbers have finite precision (of roughly 1 part in $10^{-16}$), there comes a point where increasing $N$ no longer provides a benefit. For the Trapezoidal rule, that point is $N\sim 10^{8}$ for Simpson's rule, it's $N\sim 10^{4}$.
- To estimate error, re-run integrator with double the points. Call the original answer $I_1$ and the new answer $I_2$. Then for the trapezoidal rule, the error of $I_2$ is of order
$$\epsilon_2 = \tfrac13(I_2-I_1)\,.$$
With Simpson's rule, the error is
$$\epsilon_2 = \tfrac{1}{15}(I_2-I_1)\,.$$

## Your Turn

Use the trapezoidal rule function you wrote above with 10 steps to evaluate the integral
$$\int_0^2(x^4-2x+1)\,dx\,.$$ Repeat with 20 steps, and use the two results to calculate the error (for the 20-step value). Save the 20-step integration in a varibale named `I2` and the error in a variable named `err`.

(The exact value is 4.4. Why do you think I2 + err $\ne$ 4.4?)

In [ ]:
%%graded # 1 point

# YOUR CODE HERE

In [ ]:
%%tests

_test.code_contains("trapezoidal")
_test.similar(I2, 4.43)
_test.similar(abs(err), 0.0340)

# Gaussian Quadrature

In [ ]:
%video TgmC7ZpZSZc

Summary:

- So far, we've used variations of fitting a low-order Taylor expansion to the integrand to calculate the area beneath the curve. With $N$ equally-spaced points, we could in principle fit a degree $N$ polynomial to our integrand.
- If we allow the spacing between points to be variable, we double the available degrees of freedom in our fit. With $N$ points, we can fit a polynomial of degree $2N-1$.
- The hard part is finding the ideal sampling points $x_k$ and their weights $w_k$. In brief, we expand $f(x)$ in terms of the Legendre polynomials $P_N(x)$. The $x_k$ are located at the zeros of the $N$th Legendre polynomial, and the $w_k$ are calculated using the first derivative of the same Legendre polynomial.
- `scipy.integrate.quad(func, lower, upper)` is the go-to integrator. Behind the scenes it uses a close cousin of Gaussian Quadrature called [Clenshaw-Curtis quadrature](https://en.wikipedia.org/wiki/Clenshaw%E2%80%93Curtis_quadrature). It's main advantage is that it's much simpler to estimate the error than with Guassian quadrature.

## Your Turn

The normal distribution is given by
$$\frac{1}{\sqrt{2\pi}}e^{-x^2/2}\,.$$
Use `integrate.quad` to integrate the normal distribution from 0 to $\infty$. Store the result in a variable named `I`. You can use `np.inf` for infinity.

In [ ]:
%%graded # 1 point

# YOUR CODE HERE

In [ ]:
%%tests

_test.code_contains("quad", "np.inf")
_test.similar(I, 0.5)

# Multiple Integrals

In [ ]:
%video 22v8NtaCeAo

Summary:

Three approaches for $I = \iint f(x,y)\,dx\,dy$:
1. Let
$$F(y) = \int f(x,y)\,dx$$
and
$$I = \int F(y)\,dy\,.$$
2. Write a custom integrator (especially if more than 3-D). Monte Carlo works well here.
3. Use `scipy.integrate.dblquad` (or `tplquad` for 3-D).

## Your Turn

The area of a unit circle can be calculated using integration
$$A = \int_{-1}^1 \int_{-\sqrt{1-x^2}}^\sqrt{1-x^2}\,dydx\,.$$
Use `dblquad` to evaluate this integral and store the answer in a variable named `A`. (The answer should be $\pi$.)

In [ ]:
%%graded # 1 point

# YOUR CODE HERE

In [ ]:
%%tests

_test.code_contains("dblquad")
_test.similar(A, np.pi)

# Additional Resources

- Textbook sections 5.1 to 5.9 and Appendix C - especially if you want to see a more rigorous derivation of Gaussian quadrature.